In [3]:
 %pip install -q langchain langchain_experimental genai python-dotenv langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.9/831.9 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.3.28 requires openai<2.0.0,>=1.86.0, but you have openai 0.27.10 which is incompatible.
langchain-openai 0.3.28 requires tiktoken<1,>=0.7, but you have tiktoken 0.3.3 which is incompatible.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.37

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

Load environment variables and initialize the language model

In [6]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", max_tokens=1000, temperature=0)

Create a simple in-memory store for chat histories

In [10]:
store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

Create the prompt template

In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an intelligent and helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

Combine the prompt and model into a runnable chain

In [12]:
chain = prompt | llm

Wrap the chain with message history

In [13]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

Use case

In [20]:
session_id = "user_123"


response1 = chain_with_history.invoke(
    {"input": "Hello! ?"},
    config={"configurable": {"session_id": session_id}}
)
print("### AI Response 1:\n")
print(response1.content)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("\n### AI Response 2:\n")
print(response2.content)

### AI Response 1:

Hello! How can I help you today?

### AI Response 2:

Your previous message was "Hello! ?"


In [23]:
print("\n### Conversation History:\n")
for message in store[session_id].messages:
    print(f"*{message.type}:*{message.content}\n")


### Conversation History:

*human:*Hello! ?

*ai:*Hello! How can I help you today?

*human:*What was my previous message?

*ai:*As a large language model, I have no memory of past conversations. Therefore, I don't know what your previous message was.

*human:*Hello! ?

*ai:*Hello! How can I help you today?

*human:*What was my previous message?

*ai:*Your previous message was "Hello! ?"



In [24]:
session_id = "user_1234"


response1 = chain_with_history.invoke(
    {"input": "Hello! who are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("### AI Response 1:\n")
print(response1.content)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("\n### AI Response 2:\n")
print(response2.content)

### AI Response 1:

I am a large language model, trained by Google.

### AI Response 2:

Your previous message was "Hello! who are you?".


In [26]:
print("\n### Conversation History:\n")
for message in store[session_id].messages:
    print(f"*{message.type}:*{message.content}\n")


### Conversation History:

*human:*Hello! who are you?

*ai:*I am a large language model, trained by Google.

*human:*What was my previous message?

*ai:*Your previous message was "Hello! who are you?".

*human:*Hello! who are you?

*ai:*I am a large language model, trained by Google.

*human:*What was my previous message?

*ai:*Your previous message was "Hello! who are you?".



In [27]:
session_id = "user_12345"


response1 = chain_with_history.invoke(
    {"input": "Hello! What's the highest mountain in the world and where is it located?"},
    config={"configurable": {"session_id": session_id}}
)
print("### AI Response 1:\n")
print(response1.content)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("\n### AI Response 2:\n")
print(response2.content)

### AI Response 1:

Hello! As I mentioned before, the highest mountain in the world is **Mount Everest**, located in the **Himalayas** on the border between **Nepal** and **China (Tibet Autonomous Region)**.

### AI Response 2:

Your previous message was: "Hello! What's the highest mountain in the world and where is it located?"


In [28]:
session_id = "user_12345"


response1 = chain_with_history.invoke(
    {"input": "Hello!? Can you give me a brief history about Mount Everest"},
    config={"configurable": {"session_id": session_id}}
)
print("### AI Response 1:\n")
print(response1.content)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("\n### AI Response 2:\n")
response2.content

### AI Response 1:

Hello! I can certainly give you a brief history of Mount Everest again. Here it is:

*   **Early History & Naming:** Known locally as Chomolungma (Tibetan) and Sagarmatha (Nepali). In 1856, the Great Trigonometrical Survey of India identified it as the highest peak and named it after Sir George Everest.

*   **Early Attempts:** Began in the 1920s, primarily by British expeditions from Tibet. George Mallory and Andrew Irvine disappeared in 1924.

*   **First Ascent:** May 29, 1953, Sir Edmund Hillary and Tenzing Norgay reached the summit as part of a British expedition.

*   **Later Ascents & Routes:** New routes were established, and the South Col route (via Nepal) became the most popular.

*   **Commercialization & Challenges:** Increased commercialization has led to overcrowding, environmental concerns, and safety issues.

*   **Modern Era:** Hundreds attempt to summit each year. Challenges remain related to climate change, waste, and tourism.

### AI Response 2:


'Your previous message was: "Hello!? Can you give me a brief history about Mount Everest"'